In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn as sk
import sklearn.datasets

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from pandas import DataFrame

In [3]:
df_mat = pd.read_csv('student-mat.csv')
df_por = pd.read_csv('student-por.csv')
df_both = pd.concat([df_mat, df_por])